# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import gmaps.datasets


# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
city_vac_df = pd.read_csv("../output_data/city_data1.csv")
city_vac_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,benito juarez,75,MX,1587428189,34,19.39,-99.17,80.60,17.22
1,1,hobart,40,AU,1587427898,66,-42.88,147.33,55.00,9.17
2,2,rikitea,24,PF,1587428092,71,-23.12,-134.97,77.13,9.42
3,3,santiago,69,CL,1587428102,23,-33.46,-70.65,77.00,8.05
4,4,victoria,20,HK,1587428114,74,22.29,114.16,81.00,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
city_coord = city_vac_df[["Lat", "Lng"]]
city_humid = city_vac_df['Humidity']


fig = gmaps.figure()

# Create heat layer
heat_layer = (gmaps.heatmap_layer(locations=city_coord, weights=city_humid, dissipating=False, max_intensity=10, point_radius=1)) 
                                   
# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
fav_cities_df = city_vac_df.copy()
fav_cities_df = fav_cities_df[fav_cities_df['Max Temp'] > 70]
fav_cities_df = fav_cities_df[fav_cities_df['Max Temp'] < 85]  

fav_cities_df = fav_cities_df[fav_cities_df['Cloudiness'] < 50]
fav_cities_df = fav_cities_df[fav_cities_df['Humidity'] < 50]

fav_cities_df = fav_cities_df.dropna()

fav_cities_df = fav_cities_df.reset_index()
fav_cities_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,82,82,paithan,0,IN,1587428240,20,19.48,75.38,75.67,4.94
1,134,134,tessalit,0,ML,1587428103,14,20.20,1.01,79.52,12.50
2,153,153,mungeli,28,IN,1587428289,27,22.07,81.68,80.08,0.74
3,194,194,kahului,40,US,1587428160,45,20.89,-156.47,84.20,6.93
4,195,195,hun,0,LY,1587428329,24,29.13,15.95,75.06,8.16
5,211,211,yulara,40,AU,1587428108,49,-25.24,130.99,73.40,8.05
6,234,234,werda,27,BW,1587428356,25,-25.27,23.28,72.86,4.47
7,239,239,nata,2,PA,1587428359,46,8.33,-80.52,79.00,8.63
8,244,244,saint george,1,US,1587428003,25,37.10,-113.58,75.00,6.93
9,302,302,jalu,0,LY,1587428109,17,29.03,21.55,77.61,15.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = fav_cities_df
hotel_df["Hotel Name"] = ""
hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,82,82,paithan,0,IN,1587428240,20,19.48,75.38,75.67,4.94,
1,134,134,tessalit,0,ML,1587428103,14,20.20,1.01,79.52,12.50,
2,153,153,mungeli,28,IN,1587428289,27,22.07,81.68,80.08,0.74,
3,194,194,kahului,40,US,1587428160,45,20.89,-156.47,84.20,6.93,
4,195,195,hun,0,LY,1587428329,24,29.13,15.95,75.06,8.16,
5,211,211,yulara,40,AU,1587428108,49,-25.24,130.99,73.40,8.05,
6,234,234,werda,27,BW,1587428356,25,-25.27,23.28,72.86,4.47,
7,239,239,nata,2,PA,1587428359,46,8.33,-80.52,79.00,8.63,
8,244,244,saint george,1,US,1587428003,25,37.10,-113.58,75.00,6.93,
9,302,302,jalu,0,LY,1587428109,17,29.03,21.55,77.61,15.05,


In [35]:
#search googleplaces for hotels within 5000 meters

city_rec_ctr = 0
hotels_array = []

# loop  through the city coordinates pulling hotels in the city area

for cities in hotel_df['City']:
        city_coord = str(hotel_df['Lat'][city_rec_ctr])+' , '+str(hotel_df['Lng'][city_rec_ctr])
    
        #check in a radius of 5000 meters
        city_radius = 5000
        search_type = "lodging"
        city_rec_ctr +=1
     
        params = {"location": city_coord, "radius": city_radius, "type": search_type, "key": g_key}
    
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
        try:
            response = requests.get(base_url, params=params)
            hotel_list = response.json()
        
            hotels_array.append(hotel_list['results'][0]['name'])
        
        except IndexError:
            hotels_array.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels_array
hotel_df.head()

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,82,82,paithan,0,IN,1587428240,20,19.48,75.38,75.67,4.94,Mahesh Tea Center
1,134,134,tessalit,0,ML,1587428103,14,20.20,1.01,79.52,12.50,NaN
2,153,153,mungeli,28,IN,1587428289,27,22.07,81.68,80.08,0.74,Hotel Puneet
3,194,194,kahului,40,US,1587428160,45,20.89,-156.47,84.20,6.93,Maui Seaside Hotel
4,195,195,hun,0,LY,1587428329,24,29.13,15.95,75.06,8.16,Alrwasi Hotel


In [36]:
#clean up DataSet by remooving NAN
hotel_df_clean = hotel_df.dropna()
hotel_df_clean


,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,82,82,paithan,0,IN,1587428240,20,19.48,75.38,75.67,4.94,Mahesh Tea Center
2,153,153,mungeli,28,IN,1587428289,27,22.07,81.68,80.08,0.74,Hotel Puneet
3,194,194,kahului,40,US,1587428160,45,20.89,-156.47,84.20,6.93,Maui Seaside Hotel
4,195,195,hun,0,LY,1587428329,24,29.13,15.95,75.06,8.16,Alrwasi Hotel
5,211,211,yulara,40,AU,1587428108,49,-25.24,130.99,73.40,8.05,Desert Gardens Hotel - Ayers Rock Resort
7,239,239,nata,2,PA,1587428359,46,8.33,-80.52,79.00,8.63,Cabañas Estancia Bayamon
8,244,244,saint george,1,US,1587428003,25,37.10,-113.58,75.00,6.93,Best Western Plus Abbey Inn
9,302,302,jalu,0,LY,1587428109,17,29.03,21.55,77.61,15.05,Jalu Hotel
10,330,330,khairpur nathan shah,3,PK,1587428417,25,27.09,67.73,82.47,2.35,Asad Joyo
13,395,395,cabo san lucas,5,MX,1587428049,34,22.89,-109.91,82.40,17.22,Hotel Tesoro Los Cabos


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))